# Dataset-ə qısa baxış (SAFE vs UNSAFE)

Bu notebook-un məqsədi uzun-uzadı analiz deyil :D sadəcə:
- hansı datasetləri istifadə etdiyimi,
- hansı sahədən text götürdüyümü,
- SAFE tərəfdə PII-ya oxşar cümlələri niyə filtr etdiyimi
qısa və praktik şəkildə göstərməkdir.

UNSAFE: `LocalDoc/pii_ner_azerbaijani`  
SAFE: `aznlp/azerbaijani-blogs`


In [1]:
from pathlib import Path
import os, sys, subprocess

ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

os.chdir(ROOT)
print("Repo root:", Path.cwd())

PYTHONPATH = str(Path.cwd() / "src")
ENV = os.environ.copy()
ENV["PYTHONPATH"] = PYTHONPATH + (os.pathsep + ENV["PYTHONPATH"] if ENV.get("PYTHONPATH") else "")

def run(cmd):
    """Always run with the same python as this notebook kernel + correct PYTHONPATH."""
    if isinstance(cmd, str):
        cmd = cmd.split()
    print("RUN:", " ".join(cmd))
    return subprocess.run(cmd, env=ENV, check=True)

Repo root: d:\github_repos\kontakt_home_task\task3


### Bu blok nə üçündür?

Notebook-u bəzən `task3/notebooks/` qovluğundan açırıq və o zaman `scripts/`, `data/`, `models/` kimi *relative path*lər qırılır.

Bu hissə ona görə:
- `ROOT`-u repo kökünə (task3) gətirir və `os.chdir(ROOT)` edir
- habelə `PYTHONPATH=src` əlavə edir ki, **`pii_guard` modulu pip install etmədən də** görünə bilsin
- həmçinin `run(...)` helper-i yaradır ki, subprocess çağırışları həmişə **bu notebook-un python interpreter-i** ilə işləsin (PATH-dakı başqa `python` ilə yox)

Bu, xüsusən Windows + Jupyter mühitində “niyə import tapmır?” problemlərinin qarşısını alır.

In [3]:
import re
from datasets import load_dataset

def pick_text_field(example: dict) -> str:
    for key in ("text", "content", "body", "article", "document", "sentence"):
        if key in example and isinstance(example[key], str) and example[key].strip():
            return example[key]
    for _, v in example.items():
        if isinstance(v, str) and v.strip():
            return v
    raise KeyError(f"No obvious text field found. Keys: {list(example.keys())}")

SENT_SPLIT = re.compile(r"(?<=[\.!\?…])\s+")
def split_sentences(text: str):
    text = re.sub(r"\s+", " ", text).strip()
    return [s.strip() for s in SENT_SPLIT.split(text) if s.strip()]

FIN_RE   = re.compile(r"\b\d{2}[A-Z]{2,3}[A-Z0-9]{4,6}\b")  # nümunə: 94FMDDD
PHONE_RE = re.compile(r"(\+994\s?\d{2}\s?\d{3}\s?\d{2}\s?\d{2})|(\b0\d{2}\s?\d{3}\s?\d{2}\s?\d{2}\b)")
CARD_RE  = re.compile(r"\b(?:\d[ -]*?){13,19}\b")

def looks_like_pii(s: str) -> bool:
    return bool(FIN_RE.search(s) or PHONE_RE.search(s) or CARD_RE.search(s))


## Dataset-ləri yükləyək (yalnız sütunlar + 1 nümunə)

In [4]:
unsafe = load_dataset("LocalDoc/pii_ner_azerbaijani", split="train")
safe = load_dataset("aznlp/azerbaijani-blogs", split="train")

print("UNSAFE rows:", len(unsafe))
print("UNSAFE columns:", unsafe.column_names)
print("UNSAFE sample keys:", list(unsafe[0].keys())[:10])

ex0 = safe[0]
print("\nSAFE rows:", len(safe))
print("SAFE columns:", safe.column_names)
print("SAFE text preview:", pick_text_field(ex0)[:250], "...")


UNSAFE rows: 120634
UNSAFE columns: ['uid', 'translated_text', 'privacy_mask']
UNSAFE sample keys: ['uid', 'translated_text', 'privacy_mask']

SAFE rows: 6929
SAFE columns: ['Unnamed: 0', 'url', 'title', 'author', 'category', 'date', 'content', 'tags']
SAFE text preview: 
Proqramçılığa başlamaq üçün ilk addım bu sahənin əsasları ilə tanış olmaqdır. Kompüter elmləri alqoritmlər, verilənlər strukturları və proqramlaşdırma prinsipləri ilə tanışlıq proqramçı kimi inkişafınızda əsas təşkil edəcək. Bu mərhələdə Python, Jav ...


#### **Not:** UNSAFE dataset-in real ölçüsü 120 634 sətirdir. Biz isə training zamanı sürət və CPU məhdudiyyətinə görə yalnız 8 000 sətirlik subset istifadə etmişik.

### SAFE tərəfdən cümlə çıxarışı + PII-like filtr nümunəsi

In [7]:
candidates = []
for i in range(min(60, len(safe))):
    txt = pick_text_field(safe[i])
    for s in split_sentences(txt):
        if 10 <= len(s) <= 240:
            candidates.append(s)

flagged = [s for s in candidates if looks_like_pii(s)]
print("Extracted sentences:", len(candidates))
print("Flagged as PII-like:", len(flagged))

if flagged:
    print("\nFlagged examples (first 5):")
    for s in flagged[:5]:
        print("-", s)
else:
    print("No PII-like examples in this small sample.")


Extracted sentences: 4148
Flagged as PII-like: 1

Flagged examples (first 5):
- ISBN 9 789952 835946


### Bu hissənin məqsədi nədir?
Burada etdiyimiz iş training set yaratmaq deyil. Sadəcə sanity-check edirik.

- SAFE datasetdən kiçik bir nümunə götürürük (məsələn ilk 60 yazı)
- cümlələrə bölürük
- `looks_like_pii` ilə PII-ya oxşar cümlələr varsa, onların sayına / nümunəsinə baxırıq

Əsas training dataseti yaradan real skript isə [**`scripts/build_train_classifier_json.py`**](../scripts/build_train_classifier_json.py) skriptidir.

## Qısa yekun

- Guardrail üçün data balansı vacibdir: `unsafe_count == safe_count`.
- CPU mühitində training “sonsuz” uzanmasın deyə, praktik olaraq `--max_unsafe` ilə subset götürürəm.
- Tam dataset-lə də işləmək mümkündür; sadəcə laptop/CPU üçün çox vaxt aparır.
